<a href="https://colab.research.google.com/github/hammaad2002/ASRAdversarialAttacks/blob/main/Tutorial(How_to_use_this_repository).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!git clone https://github.com/hammaad2002/ASRAdversarialAttacks.git
!git clone https://github.com/hammaad2002/CRDNN_Model.git
%cd /content/ASRAdversarialAttacks/
!pip install -r requirements.txt

In [2]:
from ASRAdversarialAttacks.AdversarialAttacks import ASRAttacks
import torchaudio
import torch
import numpy as np
from IPython.display import Audio

In [3]:
# Loading the model from torchaudio model hub
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model()

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth
100%|██████████| 360M/360M [00:04<00:00, 82.9MB/s]


In [4]:
# Checking the device available during the current environment (CUDA is recommended!)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [5]:
# Loading the audio
input_audio, sample_rate = torchaudio.load('/content/CRDNN_Model/AudioSamplesASR/spk1_snt1.wav')

In [6]:
# My target
target_transcription = 'WHEN MAN NOTHING KILL IN BIG CAMEL'
true_transcription = 'THE CHILD ALMOST HURT THE SMALL DOG'
attack = ASRAttacks(model, device, bundle.get_labels())
target = list(target_transcription.upper().replace(" ", "|"))

In [7]:
#initializing attack class
attack = ASRAttacks(model, device, bundle.get_labels())

# <center>**FGSM ATTACK**</center>

```
        Simple fast gradient sign method attack is implemented which is the simplest
        adversarial attack in this domain.
        For more information, see the paper: https://arxiv.org/pdf/1412.6572.pdf

        INPUT ARGUMENTS:

        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor

        target        : Target transcription (needed if the you want targeted 
                        attack) Ex: ["my name is mango."]. 
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the 
                        dictionary of the model also.

        epsilon       : Noise controlling parameter.
                        Type: float

        targeted      : If the attack should be targeted towards your desired 
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted 
                        transcription also in this case).

        RETURNS:

        np.ndarray : Perturbed audio
```


# **FGSM TARGETED**

In [8]:
#FGSM
temp1 = attack.FGSM_ATTACK(input_audio, target, epsilon = 0.01, targeted = True)

#FGSM PRINT
print(attack.INFER(torch.from_numpy(temp1)).replace("|"," "))
print(target_transcription)

TE CHILD ALMOST HURT A SMALL GOG 
WHEN MAN NOTHING KILL IN BIG CAMEL


For WER use [0] and for in depth information of insertion, selection, and deletion use [1]

In [9]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp1], targeted= True)[0])

Targeted WER is:  0.0


In [10]:
info = attack.wer_compute([target_transcription], [temp1], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [11]:
Audio(temp1, rate =16000)

# **FGSM UNTARGETED**

In [12]:
#FGSM
temp2 = attack.FGSM_ATTACK(input_audio, epsilon = 0.0065, targeted = False)

#FGSM PRINT
print(attack.INFER(torch.from_numpy(temp2)).replace("|"," "))

THE CHILD ALMOST HURT  THE SMALL DOG 


In [13]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp2], targeted= False)[0])

Untargeted WER is:  1.0


In [14]:
info = attack.wer_compute([target_transcription], [temp2], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [15]:
Audio(temp2, rate =16000)

# <center>**BIM ATTACK**</center>

```
        Basic Itertive Moment attack is implemented which is simple Fast Gradient 
        Sign Attack but in loop. 
        For more information, see the paper: https://arxiv.org/abs/1607.02533

        INPUT ARGUMENTS:

        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor

        target        : Target transcription (needed if the you want targeted 
                        attack) Ex: ["my name is mango."]. 
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the 
                        dictionary of the model also.

        epsilon       : Noise controlling parameter.
                        Type: float

        alpha         : Noise contribution in input controlling parameter
                        Type: float

        num_iter      : Number of iteration of attack
                        Type: int

        targeted      : If the attack should be targeted towards your desired 
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted 
                        transcription also in this case).

        early_stop    : If user wants to stop the attack early if the attack reaches the target transcription before the total number of iterations.
                        Type: bool
                        
        RETURNS:
        
        np.ndarray : Perturbed audio
```


# **BIM TARGETED**

With early stopping:

In [16]:
#BIM
temp3 = attack.BIM_ATTACK(input_audio, target, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 3000, targeted = True, early_stop = True)

#BIM PRINT
print('\n',attack.INFER(torch.from_numpy(temp3)).replace("|"," "))
print(target_transcription)

  0%|          | 0/3000 [00:00<?, ?it/s]

Breaking for loop because targeted Attack is performed successfully !

 WHEN MAN NOTHING KILL IN BIG CAMEL
WHEN MAN NOTHING KILL IN BIG CAMEL


In [17]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp3], targeted= True)[0])

Targeted WER is:  1.0


In [18]:
info = attack.wer_compute([target_transcription], [temp3], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 0, Deletion: 0


In [19]:
Audio(temp3, rate = 16000)

Without early stopping:

In [20]:
#BIM
temp4 = attack.BIM_ATTACK(input_audio, target, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 3000, targeted = True, early_stop = False)

#BIM PRINT
print('\n',attack.INFER(torch.from_numpy(temp4)).replace("|"," "))
print(target_transcription)

  0%|          | 0/3000 [00:00<?, ?it/s]


 WHEN MAN NOTHING KILL IN BIG CAMEL
WHEN MAN NOTHING KILL IN BIG CAMEL


In [21]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp4], targeted= True)[0])

Targeted WER is:  1.0


In [22]:
info = attack.wer_compute([target_transcription], [temp4], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 0, Deletion: 0


In [23]:
Audio(temp4, rate = 16000)

# **BIM UNTARGETED**

With early stopping:

In [24]:
#BIM
temp5 = attack.BIM_ATTACK(input_audio, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 2500, targeted = False, early_stop = True)

#BIM PRINT
print('\n',attack.INFER(torch.from_numpy(temp5)).replace("|"," "))

  0%|          | 0/2500 [00:00<?, ?it/s]

Breaking for loop because untargeted Attack is performed successfully !

 CHILD ALMOST HURT THE SMALL DOG 


In [25]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp5], targeted= False)[0])

Untargeted WER is:  1.0


In [26]:
info = attack.wer_compute([target_transcription], [temp5], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 6, Deletion: 1


In [27]:
Audio(temp5, rate =16000)

Without early stopping:

In [28]:
#BIM
temp6 = attack.BIM_ATTACK(input_audio, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 2500, targeted = False, early_stop = False)

#BIM PRINT
print('\n',attack.INFER(torch.from_numpy(temp6)).replace("|"," "))

  0%|          | 0/2500 [00:00<?, ?it/s]


 OWIM IKMIWL KWA UKMIUIKIWLI TIOTUIEUINIEKNWMU MUON NKIMIOMU ILIMINO WKUKNIM


In [29]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp6], targeted= False)[0])

Untargeted WER is:  1.0


In [30]:
info = attack.wer_compute([target_transcription], [temp6], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 2, Substitution: 7, Deletion: 0


In [31]:
Audio(temp6, rate =16000)

# <center>**PGD ATTACK**</center>

```
        Projected Gradient Descent attack is implemented which in simple terms is more 
        advanced version of BIM. In this attack we project our perturbation back to 
        some Lp norm and find perturbations in that particular region. 
        For more information, see the paper: https://arxiv.org/abs/1706.06083

        INPUT ARGUMENTS:

        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor

        target        : Target transcription (needed if the you want targeted 
                        attack) Ex: ["my name is mango."]. 
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the 
                        dictionary of the model also.

        epsilon       : Noise controlling parameter.
                        Type: float

        alpha         : Noise contribution in input controlling parameter
                        Type: float

        num_iter      : Number of iteration of attack
                        Type: int

        targeted      : If the attack should be targeted towards your desired 
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted 
                        transcription also in this case).

        early_stop    : If user wants to stop the attack early if the attack reaches the target transcription before the total number of iterations.
                        Type: bool
                        
        RETURNS:
        
        np.ndarray : Perturbed audio
```

# **PGD TARGETED**

With early stopping:

In [32]:
#PGD
temp7 = attack.PGD_ATTACK(input_audio, target, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 2500, targeted = True, early_stop = True)

#PGD PRINT
print('\n',attack.INFER(torch.from_numpy(temp7)).replace("|"," "))
print(target_transcription)

  0%|          | 0/2500 [00:00<?, ?it/s]

Breaking for loop because targeted Attack is performed successfully !

 WHEN MAN NOTHING KILL IN BIG CAMEL
WHEN MAN NOTHING KILL IN BIG CAMEL


In [33]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp7], targeted= True)[0])

Targeted WER is:  1.0


In [34]:
info = attack.wer_compute([target_transcription], [temp7], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 0, Deletion: 0


In [35]:
Audio(temp7, rate =16000)

Without early stopping:

In [36]:
#PGD
temp8 = attack.PGD_ATTACK(input_audio, target, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 2500, targeted = True, early_stop = False)

#PGD PRINT
print('\n',attack.INFER(torch.from_numpy(temp8)).replace("|"," "))
print(target_transcription)

  0%|          | 0/2500 [00:00<?, ?it/s]


 WHEN MAN NOTHING ILL IN BIG CAML
WHEN MAN NOTHING KILL IN BIG CAMEL


In [37]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp8], targeted= True)[0])

Targeted WER is:  0.7142857142857143


In [38]:
info = attack.wer_compute([target_transcription], [temp8], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 2, Deletion: 0


In [39]:
Audio(temp8, rate =16000)

# **PGD UNTARGETED**

With early stopping:

In [40]:
#PGD
temp9 = attack.PGD_ATTACK(input_audio, epsilon = 0.0015, alpha = 0.0001,
                   num_iter = 10000, targeted = False, early_stop = True)

#PGD PRINT
print('\n',attack.INFER(torch.from_numpy(temp9)).replace("|"," "))

  0%|          | 0/10000 [00:00<?, ?it/s]


 THE CHILD ALMOST  HRIRKEDD  THE SMAALL DOOG 


In [41]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp9], targeted= False)[0])

Untargeted WER is:  1.0


In [42]:
info = attack.wer_compute([target_transcription], [temp9], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [43]:
Audio(temp9, rate =16000)

Without early stopping:

In [44]:
#PGD
temp10 = attack.PGD_ATTACK(input_audio, epsilon = 0.0015, alpha = 0.0001,
                   num_iter = 10000, targeted = False, early_stop = True)

#PGD PRINT
print('\n',attack.INFER(torch.from_numpy(temp10)).replace("|"," "))

  0%|          | 0/10000 [00:00<?, ?it/s]

Breaking for loop because untargeted Attack is performed successfully !

 IT A CHILD ALMOST CCURTS OF A SMALL DOG  


In [45]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp10], targeted= False)[0])

Untargeted WER is:  1.0


In [46]:
info = attack.wer_compute([target_transcription], [temp10], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 2, Substitution: 7, Deletion: 0


In [47]:
Audio(temp10, rate =16000)

# <center>**CW ATTACK**</center>

```
        Implements the Carlini and Wagner attack, the strongest white box 
        adversarial attack. This attack uses an optimization strategy to find the 
        adversarial transcription while keeping the perturbation as low as possible. 
        For more information, see the paper: https://arxiv.org/pdf/1801.01944.pdf
        
        INPUT ARGUMENTS:
        
        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor
                        
        target        : Target transcription (needed if the you want targeted 
                        attack) Ex: ["my name is mango."]. 
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the 
                        dictionary of the model also.
                        
        epsilon       : Noise controlling parameter.
                        Type: float
                        
        c             : Regularization term controlling factor.
                        Type: float
                        
        learning_rate : learning_rate of optimizer.
                        Type: float
                        
        num_iter      : Number of iteration of attack.
                        Type: int
                        
        decrease_factor_eps   : Factor to decrease epsilon during search
                                Type: float
                                
        num_iter_decrease_eps : Number of iterations after which to decrease epsilon
                                Type: int
                                
        optimizer     : Name of the optimizer to use for the attack. 
                        Type: str
                        
        nested        : if this attack in being run in a for loop with tqdm 
                        Type: bool
                        
        early_stop    : if the user wants to end the attack as soon as the attack
                        gets the target transcription.
                        Type: bool
                        
        search_eps    : if the user wants the attack to search for the epsilon value
                        on its own provided the initial epsilon value of large.
                        Type: bool
        
        targeted      : If the attack should be targeted towards your desired 
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted 
                        transcription also in this case).
                        
        RETURNS:
        
        np.ndarray : Perturbed audio
```

# **CW TARGETED**

With early stopping:

In [48]:
#CW
temp11 = attack.CW_ATTACK(input_audio, target, epsilon = 0.0015, c = 10,
                  learning_rate = 0.00001, num_iter = 10000, decrease_factor_eps = 1,
                  num_iter_decrease_eps = 10, optimizer = None, nested = True, 
                  early_stop = True, search_eps = False, targeted = True)

#CW PRINT
print('\n',attack.INFER(torch.from_numpy(temp11)).replace("|"," "))
print(target_transcription)

  0%|          | 0/10000 [00:00<?, ?it/s]


 CHIL MAN NOTHING KILL IN BIG CAMEL
WHEN MAN NOTHING KILL IN BIG CAMEL


In [49]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp11], targeted= True)[0])

Targeted WER is:  0.8571428571428572


In [50]:
info = attack.wer_compute([target_transcription], [temp11], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 1, Deletion: 0


In [51]:
Audio(temp11, rate =16000)

Without early stopping:

In [52]:
#CW
temp12 = attack.CW_ATTACK(input_audio, target, epsilon = 0.0015, c = 10,
                  learning_rate = 0.00001, num_iter = 10000, decrease_factor_eps = 1,
                  num_iter_decrease_eps = 10, optimizer = None, nested = True, 
                  early_stop = False, search_eps = False, targeted = True)

#CW PRINT
print('\n',attack.INFER(torch.from_numpy(temp12)).replace("|"," "))
print(target_transcription)

  0%|          | 0/10000 [00:00<?, ?it/s]


 CHIL MAN NOTHING KILL IN BIG CAMEL
WHEN MAN NOTHING KILL IN BIG CAMEL


In [53]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp12], targeted= True)[0])

Targeted WER is:  0.8571428571428572


In [54]:
info = attack.wer_compute([target_transcription], [temp12], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 1, Deletion: 0


In [55]:
Audio(temp12, rate =16000)

# **CW UNTARGETED**

With early stopping:

In [56]:
#CW
temp13 = attack.CW_ATTACK(input_audio, epsilon = 0.0015, c = 10,
                  learning_rate = 0.00001, num_iter = 10000, decrease_factor_eps = 1,
                  num_iter_decrease_eps = 10, optimizer = None, nested = True,
                  early_stop = True, search_eps = False, targeted = False)

#CW PRINT
print('\n',attack.INFER(torch.from_numpy(temp13)).replace("|"," "))

  0%|          | 0/10000 [00:00<?, ?it/s]

Breaking for loop because targeted Attack is performed successfully !

 THECHILD ALMOST HURT HE SMALL DOG  


In [57]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp13], targeted= False)[0])

Untargeted WER is:  1.0


In [58]:
info = attack.wer_compute([target_transcription], [temp13], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 6, Deletion: 1


In [59]:
Audio(temp13, rate =16000)

Without early stopping:

In [60]:
#CW
temp14 = attack.CW_ATTACK(input_audio, epsilon = 0.0015, c = 10,
                  learning_rate = 0.00001, num_iter = 10000, decrease_factor_eps = 1,
                  num_iter_decrease_eps = 10, optimizer = None, nested = True,
                  early_stop = False, search_eps = False, targeted = False)

#CW PRINT
print('\n',attack.INFER(torch.from_numpy(temp14)).replace("|"," "))

  0%|          | 0/10000 [00:00<?, ?it/s]


  EY E EIELEWE ONAOAWLAMONWASTIEOE EOIKIKYEIOWO I TINM LOENMEDO NOENOEKE


In [61]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp14], targeted= False)[0])

Untargeted WER is:  1.0


In [62]:
info = attack.wer_compute([target_transcription], [temp14], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 2, Substitution: 7, Deletion: 0


In [63]:
Audio(temp14, rate = 16000)

# <center>**IMPERCEPTIBLE ASR ATTACK**</center>


```
        Implements the Imperceptible ASR attack, which leverages the strongest white box 
        adversarial attack which is CW attack while also masking sure the added perturbation
        is imperceptible to humans using Psychoacoustic Scale. This attack is performed in two 
        stages. In first stage we perform simple CW attack and in 2nd stage we make sure our 
        added perturbations are imperceptible. 
        For more information, see the paper: https://arxiv.org/abs/1903.10346
        
        INPUT ARGUMENTS:
        
        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor
                        
        target        : Target transcription (needed if the you want targeted 
                        attack) Ex: ["my name is mango."]
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the 
                        dictionary of the model also
                        
        epsilon       : Noise controlling parameter
                        Type: float
                        
        c             : Regularization term controlling factor
                        Type: float
                        
        learning_rate1: learning_rate of optimizer for stage 1
                        Type: float
        
        learning_rate2: learning_rate of optimizer for stage 2
                        Type: float
                        
        num_iter1     : Number of iteration of attack stage 1
                        Type: int
        
        num_iter2     : Number of iteration of attack stage 2
                        Type: int
                        
        decrease_factor_eps   : Factor to decrease epsilon by during optimization 
                                Type: float
                                
        num_iter_decrease_eps : Number of iterations after which to decrease epsilon
                                Type: int
                                
        optimizer1     : Name of the optimizer to use for the attack stage 1
                         Type: str
                         
        optimizer2     : Name of the optimizer to use for the attack stage 2
                         Type: str
                         
        nested         : if this attack in being run in a for loop with tqdm 
                         Type: bool
        
        early_stop_cw  : if the user wants 1st stage attack to early stop or not
                         Type: bool
        
        search_eps_cw  : if the user wants 1st stage attack to search for epsilon 
                         value provided a large intial epsilon value is provided
                         Type: bool
                         
        alpha          : regularization term for second stage loss
                         Type: float
                         
        RETURNS:
        
        np.ndarray     : Perturbed audio
```

With early stopping:

In [64]:
temp15 = attack.IMPERCEPTIBLE_ATTACK(torch.nn.functional.pad(input_audio, (0, 1000)), target, epsilon = 0.015, c = 10, learning_rate1 = 0.001, 
                                    learning_rate2 = 0.0001, num_iter1 = 1000, num_iter2 = 15000, decrease_factor_eps = 1, 
                                    num_iter_decrease_eps = 10, optimizer1 = None, optimizer2 = "Adam",nested = True , 
                                    early_stop_cw = True, search_eps_cw = False, alpha = 0.05)

# Attack's transcription
print('\n',attack.INFER(torch.from_numpy(temp15)).replace("|"," "))

***** Attack Stage 1 *****


  0%|          | 0/1000 [00:00<?, ?it/s]

Breaking for loop because targeted Attack is performed successfully !
***** Attack Stage 2 *****


  0%|          | 0/15000 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/functional.py:641: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:862.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]



 WHEN MAN NOTHING KILL IN BIG CAMEL


In [65]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp15], targeted= True)[0])

Targeted WER is:  1.0


In [66]:
info = attack.wer_compute([target_transcription], [temp15], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 0, Deletion: 0


In [67]:
Audio(temp15[:,:-1000], rate = 16000)

Without early stopping:

In [68]:
temp16 = attack.IMPERCEPTIBLE_ATTACK(torch.nn.functional.pad(input_audio, (0, 1000)), target, epsilon = 0.015, c = 10, learning_rate1 = 0.001, 
                                    learning_rate2 = 0.0001, num_iter1 = 1000, num_iter2 = 15000, decrease_factor_eps = 1, 
                                    num_iter_decrease_eps = 10, optimizer1 = None, optimizer2 = "Adam",nested = True , 
                                    early_stop_cw = False, search_eps_cw = False, alpha = 0.05)

# Attack's transcription
print('\n',attack.INFER(torch.from_numpy(temp16)).replace("|"," "))

***** Attack Stage 1 *****


  0%|          | 0/1000 [00:00<?, ?it/s]

***** Attack Stage 2 *****


  0%|          | 0/15000 [00:00<?, ?it/s]

In [69]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp16], targeted= True)[0])

Targeted WER is:  0.0


In [70]:
info = attack.wer_compute([target_transcription], [temp16], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 0, Deletion: 7


In [71]:
Audio(temp16[:,:-1000], rate = 16000)

----------------------------

# <center> **COMPARISON**

METRIC I USED FOR COMPARISON

In [72]:
# SNR metric for comparison to check which attack adds less noise
def Metricsnr(original, noisy):
    original_power = 20 * np.log10(np.mean(original ** 2))
    noise_power = 20 * np.log10(np.mean((original - noisy) ** 2))
    snr = noise_power - original_power
    return snr

In [73]:
orig = input_audio.numpy() # clean audio
orig.shape

(1, 45920)

-------------------

# **TARGETED ATTACKS**

With early stopping

In [74]:
Audio(temp3, rate = 16000) #BIM targeted

In [75]:
Audio(temp7, rate = 16000) #PGD targeted

In [76]:
Audio(temp11, rate = 16000) #CW targeted

In [77]:
Audio(temp15[:,:-1000], rate = 16000) # Imperceptible ASR

In [78]:
print("LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED")
print("BIM SNR                  :",Metricsnr(orig, temp3)) 
print("PGD SNR                  :",Metricsnr(orig, temp7)) 
print("CW SNR                   :",Metricsnr(orig, temp11)) 
print("IMPERCEPTIBLE ATTACK SNR :",Metricsnr(orig, temp15[:,:-1000]))

LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED
BIM SNR                  : -41.37290000915527
PGD SNR                  : -52.61950492858887
CW SNR                   : -56.00250244140625
IMPERCEPTIBLE ATTACK SNR : -34.916744232177734


Without early stopping

In [79]:
Audio(temp4, rate = 16000) #BIM targeted

In [80]:
Audio(temp8, rate = 16000) #PGD targeted

In [81]:
Audio(temp12, rate = 16000) #CW targeted

In [82]:
Audio(temp16[:,:-1000], rate = 16000) # Imperceptible ASR

In [83]:
print("LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED")
print("BIM SNR                  :",Metricsnr(orig, temp4)) 
print("PGD SNR                  :",Metricsnr(orig, temp8)) 
print("CW SNR                   :",Metricsnr(orig, temp12)) 
print("IMPERCEPTIBLE ATTACK SNR :",Metricsnr(orig, temp16[:,:-1000]))

LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED
BIM SNR                  : -23.13732147216797
PGD SNR                  : -52.24007606506348
CW SNR                   : -55.575008392333984
IMPERCEPTIBLE ATTACK SNR : nan


-------------------

-------------------

# **UNTARGETED ATTACKS**

With early stopping

In [84]:
Audio(temp5, rate = 16000) #BIM untargeted

In [85]:
Audio(temp9, rate = 16000) #PGD untargeted

In [86]:
Audio(temp13, rate = 16000) #CW untargeted

In [87]:
print("LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED") 
print("BIM SNR:",Metricsnr(orig, temp5)) 
print("PGD SNR:",Metricsnr(orig, temp9)) 
print("CW SNR :",Metricsnr(orig, temp13)) 

LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED
BIM SNR: -79.11591529846191
PGD SNR: -52.33769416809082
CW SNR : -92.73252487182617


Without early stopping

In [88]:
Audio(temp6, rate = 16000) #BIM untargeted

In [89]:
Audio(temp10, rate = 16000) #PGD untargeted

In [90]:
Audio(temp14, rate = 16000) #PGD untargeted

In [91]:
print("LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED") 
print("BIM SNR:",Metricsnr(orig, temp6)) 
print("PGD SNR:",Metricsnr(orig, temp10)) 
print("CW SNR :",Metricsnr(orig, temp14)) 

LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED
BIM SNR: -23.9810848236084
PGD SNR: -52.50859260559082
CW SNR : -55.923452377319336


-------------------